In [4]:
!pip install flask

In [5]:
def classify_subject(text):
    keywords = {
        "Physics": ["force", "acceleration", "newton", "velocity", "gravity", "mass"],
        "Biology": ["photosynthesis", "cell", "organism", "dna", "enzyme", "plant"],
        "Chemistry": ["atom", "molecule", "reaction", "acid", "base", "compound"],
        "Math": ["algebra", "equation", "geometry", "calculus", "derivative", "matrix"],
        "History": ["war", "revolution", "empire", "king", "dynasty", "ancient"],
    }

    text = text.lower()
    for subject, words in keywords.items():
        for word in words:
            if word in text:
                return subject
    return "General"


In [6]:
import sqlite3

def init_db():
    conn = sqlite3.connect("flashcards.db")
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS flashcards (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    student_id TEXT,
                    question TEXT,
                    answer TEXT,
                    subject TEXT
                )''')
    conn.commit()
    conn.close()

def add_flashcard(student_id, question, answer, subject):
    conn = sqlite3.connect("flashcards.db")
    c = conn.cursor()
    c.execute("INSERT INTO flashcards (student_id, question, answer, subject) VALUES (?, ?, ?, ?)",
              (student_id, question, answer, subject))
    conn.commit()
    conn.close()

def get_flashcards(student_id, limit):
    conn = sqlite3.connect("flashcards.db")
    c = conn.cursor()
    c.execute("SELECT question, answer, subject FROM flashcards WHERE student_id=?", (student_id,))
    rows = c.fetchall()
    conn.close()

    from random import shuffle
    shuffle(rows)

    seen_subjects = set()
    results = []
    for row in rows:
        if row[2] not in seen_subjects:
            results.append({
                "question": row[0],
                "answer": row[1],
                "subject": row[2]
            })
            seen_subjects.add(row[2])
        if len(results) >= limit:
            break
    return results


In [7]:
from flask import Flask, request, jsonify
import threading

app = Flask(__name__)
init_db()

@app.route("/flashcard", methods=["POST"])
def add_card():
    data = request.get_json()
    student_id = data.get("student_id")
    question = data.get("question")
    answer = data.get("answer")
    
    subject = classify_subject(question)
    add_flashcard(student_id, question, answer, subject)

    return jsonify({"message": "Flashcard added successfully", "subject": subject})

@app.route("/get-subject", methods=["GET"])
def get_cards():
    student_id = request.args.get("student_id")
    limit = int(request.args.get("limit", 5))
    cards = get_flashcards(student_id, limit)
    return jsonify(cards)

def run_app():
    app.run(port=5000)

# Start Flask in a background thread
flask_thread = threading.Thread(target=run_app)
flask_thread.daemon = True
flask_thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [8]:
import requests
import time

# Wait for Flask to start
time.sleep(2)

# Add flashcards
flashcards = [
    {"student_id": "stu001", "question": "What is Newton's Second Law?", "answer": "Force equals mass times acceleration"},
    {"student_id": "stu001", "question": "What is photosynthesis?", "answer": "A process used by plants to convert light into energy"},
    {"student_id": "stu001", "question": "What is an atom?", "answer": "The basic unit of a chemical element"},
    {"student_id": "stu001", "question": "Solve x + 2 = 5", "answer": "x = 3"},
    {"student_id": "stu001", "question": "Who was the first emperor of Rome?", "answer": "Augustus Caesar"}
]

for card in flashcards:
    r = requests.post("http://127.0.0.1:5000/flashcard", json=card)
    print(r.json())


127.0.0.1 - - [18/May/2025 15:23:25] "POST /flashcard HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2025 15:23:25] "POST /flashcard HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2025 15:23:25] "POST /flashcard HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2025 15:23:25] "POST /flashcard HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2025 15:23:25] "POST /flashcard HTTP/1.1" 200 -


{'message': 'Flashcard added successfully', 'subject': 'Physics'}
{'message': 'Flashcard added successfully', 'subject': 'Biology'}
{'message': 'Flashcard added successfully', 'subject': 'Chemistry'}
{'message': 'Flashcard added successfully', 'subject': 'General'}
{'message': 'Flashcard added successfully', 'subject': 'General'}


In [9]:
# Get flashcards (mixed subjects)
r = requests.get("http://127.0.0.1:5000/get-subject", params={"student_id": "stu001", "limit": 5})
r.json()

127.0.0.1 - - [18/May/2025 15:23:49] "GET /get-subject?student_id=stu001&limit=5 HTTP/1.1" 200 -


[{'answer': 'x = 3', 'question': 'Solve x + 2 = 5', 'subject': 'General'},
 {'answer': 'Force equals mass times acceleration',
  'question': "What is Newton's Second Law?",
  'subject': 'Physics'},
 {'answer': 'The basic unit of a chemical element',
  'question': 'What is an atom?',
  'subject': 'Chemistry'},
 {'answer': 'A process used by plants to convert light into energy',
  'question': 'What is photosynthesis?',
  'subject': 'Biology'}]